In [1]:
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

True

In [2]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_core.runnables import RunnablePassthrough
from pinecone import Pinecone, ServerlessSpec

/home/weriks/anaconda3/envs/sistema_turismo_langchain/lib/python3.13/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [3]:
# model = ChatGroq(model='llama-3.1-8b-instant',temperature=0)
model = ChatGroq(model='meta-llama/llama-4-maverick-17b-128e-instruct',temperature=0)

In [4]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings_model = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipykernel_939869/1048288091.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings_model = HuggingFaceEmbeddings(model_name=model_name)


In [5]:
pinecone_client = Pinecone()
index_name = "sistema-turismo-langchain"
index_name_exists = False

for index in pinecone_client.list_indexes():
  index_name = index['name']

  if index_name == "sistema-turismo-langchain":
    index_name_exists = True
    break

if not index_name_exists:
  pinecone_client.create_index(
    name=index_name,
    metric="cosine",
    dimension=384,
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
  )

In [6]:
base_conhecimento_paris = "./base_conhecimento/paris.txt"
base_conhecimento_rio_de_janeiro = "./base_conhecimento/rio_de_janeiro.txt"

loader_paris = TextLoader(base_conhecimento_paris)
loader_rio = TextLoader(base_conhecimento_rio_de_janeiro)

documents_paris = loader_paris.load()
documents_rio = loader_rio.load()

all_documents = documents_paris + documents_rio

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=200)

docs_split = text_splitter.split_documents(all_documents)

In [7]:
vectorstore = PineconeVectorStore.from_documents(
  documents=docs_split,
  embedding=embeddings_model,
  index_name=index_name
)

In [24]:
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs={"k": 6})

In [25]:
# Recebe a intenção do usuário e classifica em uma das categorias predefinidas
prompt_template_category_definition = PromptTemplate.from_template(
  """
    Classifique a pergunta do usuário em uma das seguintes categorias. Seja muito severo na classificação.
    Escreva apenas uma categoria. Por exemplo, se a pergunta for sobre roteiro de viagem, responda apenas "Roteiro de Viagem".
    Se a pergunta não se enquadrar em nenhuma das categorias, responda apenas "Geral".

    Categorias:
    - Roteiro de Viagem
    - Logística de Transporte
    - Informações sobre pontos turísticos
    - Guia de tradução e idiomas
    - Geral

    Pergunta: {query}
    Classificação:
  """
)

chain = (prompt_template_category_definition | model | StrOutputParser())

In [26]:
# Cadeia para extrair o perfil do usuário

prompt_extract_profile = PromptTemplate.from_template(
"""
  Analise a consulta do usuário e identifique o perfil do turista.
  Se não for possível identificar, retorne "geral".

  Perfis possíveis: aventura, cultural, gastronômico, relaxante, família.

  Consulta do usuário: {query}
  Perfil:
"""
)

profile_extraction_chain = prompt_extract_profile | model | StrOutputParser()

In [27]:
location_extraction_chain = PromptTemplate(
    input_variables=["query"],
    template="""
    Analise a consulta do usuário e identifique a localização mencionada.
    Se não for possível identificar, retorne "não especificado".

    Consulta do usuário: {query}
    Localização:
  """
)

location_extraction_chain = location_extraction_chain | model | StrOutputParser()

In [28]:
guide_to_the_assistants = """
  Sua tarefa é responder à consulta do usuário baseando-se ÚNICA E EXCLUSIVAMENTE no contexto fornecido.
  É estritamente proibido utilizar qualquer conhecimento externo ao que foi apresentado no contexto.
  Não invente informações. Não complete detalhes que não estão no texto.
  Se a resposta para a pergunta não puder ser encontrada no contexto fornecido, você deve responder exatamente:
  "Desculpe, não tenho informações sobre isso na minha base de conhecimento atual."
  Não tente adivinhar ou inferir informações.
"""

In [29]:
prompt_template_travel_itinerary = PromptTemplate(
  input_variables=["query", "profile", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer roteiros de viagem personalizados.
    Com base na consulta do usuário, crie um roteiro detalhado que inclua:
    - Principais atrações a serem visitadas
    - Eventos locais durante o período da viagem
    - Sugestões que se alinhem ao perfil do turista.

    Perfil do turista: {profile}
    Consulta do usuário: {query}
    Context: {context}
  """ % guide_to_the_assistants
) | model

In [30]:
prompt_template_transport_logistics = PromptTemplate(
  input_variables=["query", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer informações sobre logística de transporte.
    Com base na consulta do usuário, apresente um relatório logístico que inclua:
    - Opções de transporte do local
    - Sugestões de acomodação
    - Dicas práticas para facilitar a viagem

    Consulta do usuário: {query}
    Context: {context}

  """ % guide_to_the_assistants
) | model

In [31]:
prompt_template_local_information = PromptTemplate(
  input_variables=["query", "context"],
  template="""
    Guia geral: %s

    Você é um assistente especializado em fornecer informações locais detalhadas.
    Com base na consulta do usuário, forneça informações específicas sobre:
    - Pontos turísticos
    - Restaurantes recomendados
    - Horários de funcionamento e outras informações relevantes

    Consulta do usuário: {query}
    Context: {context}
    """ % guide_to_the_assistants
)

rag_chain = (
  {"context": retriever, "query": RunnablePassthrough()}
  | prompt_template_local_information
  | model
  | StrOutputParser()
)

In [32]:
prompt_template_translation_guide = PromptTemplate(
  input_variables=["query"],
  template= """
    Guia geral: %s

    Você é um assistente especializado em fornecer guias de tradução para viajantes.
    Com base na consulta do usuário, crie um guia de tradução que inclua:
    - Frases úteis no idioma local
    - Dicas culturais para facilitar a comunicação

    Consulta do usuário: {query}
    """ % guide_to_the_assistants
) | model

In [33]:
prompt_template_general = PromptTemplate(

  input_variables=["query"],

  template="""
    Guia geral: %s

    Você é um assistente de informações gerais.
    Se a consulta do usuário não se enquadrar nas categorias de roteiro de viagem,
    logística de transporte, informações locais ou guia de tradução,
    responda com uma mensagem educada informando que a consulta não pode ser atendida.
    E se a localização não estiver nas localizações conhecidas, responda "Desculpe, não tenho essa informação no momento."
    E sugira tipos de perguntas que você pode responder.

    Consulta do usuário: {query}
    """ % (guide_to_the_assistants)
) | model

In [34]:
def route(category: dict):
  data = category["category"].lower()

  if "roteiro de viagem" in data:
    return prompt_template_travel_itinerary
  elif "logística de transporte" in data:
    return prompt_template_transport_logistics
  elif "informações sobre pontos turísticos" in data:
    return prompt_template_local_information
  elif "guia de tradução e idiomas" in data:
    return prompt_template_translation_guide
  else:
    return prompt_template_general

In [35]:
# Exemplo 1: Roteiro de Viagem
query = "Estou planejando uma viagem gastronomica de 3 dias em Paris. Quais atrações você recomenda?"
final_response = rag_chain.invoke(query)
print(final_response)

Para uma viagem gastronômica de 3 dias em Paris, algumas das principais atrações recomendadas incluem:

1. **Torre Eiffel**: O monumento mais emblemático de Paris, com vistas incríveis da cidade a partir de seus três níveis. Localizada no Champ de Mars, no 7º arrondissement. O horário de funcionamento é geralmente das 9h30 às 23h, mas é recomendável comprar ingressos online com antecedência.

2. **Museu do Louvre**: O maior museu de arte do mundo, lar de obras-primas como a Mona Lisa e a Vênus de Milo. Localizado no 1º arrondissement, na margem direita do Sena. É aconselhável planejar a visita com antecedência devido ao tamanho do museu.

3. **Centre Pompidou**: Museu de arte moderna com arquitetura inovadora e exposições contemporâneas, localizado no 4º arrondissement.

Além disso, os bairros gastronômicos como o Latin Quarter (5º arrondissement) e Saint-Germain (6º arrondissement) são altamente recomendados para experiências gastronômicas.

Desculpe, não tenho informações sobre resta

In [36]:
# Exemplo 2: Logística de Transporte
query = "Quais são as melhores opções de transporte do aeroporto Charles de Gaulle para o centro de Paris?"
final_response = rag_chain.invoke(query)
print(final_response)

Olá! Para se deslocar do Aeroporto Charles de Gaulle para o centro de Paris, você tem algumas opções eficientes. O trem RER B é uma delas, conectando diretamente o aeroporto ao centro da cidade. Além disso, está previsto para 2025 que novas linhas de metrô automatizadas estarão disponíveis, o que reduzirá ainda mais o tempo de viagem. Você pode utilizar o RER B ou aguardar pelas novas opções de metrô automatizado, dependendo da sua data de viagem e preferências. É sempre uma boa ideia verificar os horários e opções de transporte disponíveis mais atualizados antes da sua viagem.
